In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

In [2]:
df=pd.read_csv("C:\\Users\DELL\\Downloads\\Chicago_Crimes_2012_to_2017.csv")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\DELL\AppData\Local\Temp\ipykernel_15176\2334669570.py:1: SyntaxWarning: invalid escape sequence '\D'
  df=pd.read_csv("C:\\Users\DELL\\Downloads\\Chicago_Crimes_2012_to_2017.csv")


In [3]:
df=pd.read_csv(r"C:\Users\DELL\Downloads\Chicago_Crimes_2012_to_2017.csv")


In [4]:
df.columns.tolist()

['Unnamed: 0',
 'ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [5]:
df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,3,10508693,HZ250496,05/03/2016 11:40:00 PM,013XX S SAWYER AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,...,24.0,29.0,08B,1154907.0,1893681.0,2016,05/10/2016 03:56:50 PM,41.864073,-87.706819,"(41.864073157, -87.706818608)"
1,89,10508695,HZ250409,05/03/2016 09:40:00 PM,061XX S DREXEL AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,...,20.0,42.0,08B,1183066.0,1864330.0,2016,05/10/2016 03:56:50 PM,41.782922,-87.604363,"(41.782921527, -87.60436317)"
2,197,10508697,HZ250503,05/03/2016 11:31:00 PM,053XX W CHICAGO AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,...,37.0,25.0,24,1140789.0,1904819.0,2016,05/10/2016 03:56:50 PM,41.894908,-87.758372,"(41.894908283, -87.758371958)"
3,673,10508698,HZ250424,05/03/2016 10:10:00 PM,049XX W FULTON ST,0460,BATTERY,SIMPLE,SIDEWALK,False,...,28.0,25.0,08B,1143223.0,1901475.0,2016,05/10/2016 03:56:50 PM,41.885687,-87.749516,"(41.885686845, -87.749515983)"
4,911,10508699,HZ250455,05/03/2016 10:00:00 PM,003XX N LOTUS AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,...,28.0,25.0,06,1139890.0,1901675.0,2016,05/10/2016 03:56:50 PM,41.886297,-87.761751,"(41.886297242, -87.761750709)"


In [9]:
# Running the ML pipeline now: RandomForest + GradientBoosting baseline, produce 60-day forecasts and RMSE summary.
# This will use the uploaded CSV at /mnt/data/Chicago_Crimes_2012_to_2017.csv
# and save outputs to /mnt/data/*.csv

import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import joblib
import matplotlib.pyplot as plt

# Helper to display DataFrame in the UI


# CONFIG
FILE_PATH = df
TOP_K = 5
WINDOW = 30            # how many past days used as features
TEST_DAYS = 365
FORECAST_HORIZON = 60
MODEL_DIR = "/mnt/data/models_ml"
os.makedirs(MODEL_DIR, exist_ok=True)

# 1) Load & detect columns
assert os.path.exists(FILE_PATH), f"Dataset not found at {FILE_PATH}"
df = pd.read_csv(FILE_PATH, low_memory=False)
# Find date column
date_col = None
for c in df.columns:
    if 'date' in c.lower():
        date_col = c
        break
if date_col is None:
    raise ValueError("No date column found. Ensure a column name contains 'date'.")
df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
df = df.dropna(subset=[date_col])
df['date'] = df[date_col].dt.floor('D')

# Use provided column names: 'Primary Type' exists
cat_col = 'Primary Type' if 'Primary Type' in df.columns else None
if cat_col is None:
    for c in df.columns:
        if c != date_col and df[c].dtype == object and df[c].nunique() < 300:
            cat_col = c; break
if cat_col is None:
    raise ValueError("No category column found.")

# 2) Aggregate into daily counts per category
daily = df.groupby(['date', df[cat_col]]).size().reset_index(name='count')
pivot = daily.pivot(index='date', columns=df[cat_col].name, values='count').fillna(0)
pivot = pivot.asfreq('D', fill_value=0)
pivot['TOTAL'] = pivot.sum(axis=1)

# 3) pick top categories
category_totals = pivot.drop(columns=['TOTAL']).sum().sort_values(ascending=False)
top_cats = list(category_totals.head(TOP_K).index)
series_names = top_cats + ['TOTAL']
ts = pivot[series_names]

# 4) train/test split
train_end = ts.index[-1] - pd.Timedelta(days=TEST_DAYS)
train = ts[ts.index <= train_end]
test = ts[ts.index > train_end]

def make_lag_features(series, window, include_calendar=True):
    X, y, idxs = [], [], []
    for i in range(window, len(series)):
        past = series.values[i-window:i].astype(float)
        row = past.copy()
        if include_calendar:
            dt = series.index[i]
            cal = np.array([dt.weekday(), dt.month, 1 if dt.weekday()>=5 else 0])
            row = np.concatenate([row, cal])
        X.append(row)
        y.append(series.values[i])
        idxs.append(series.index[i])
    return np.array(X), np.array(y), np.array(idxs)

results = []
rf_forecasts = {}
gb_forecasts = {}

for name in series_names:
    s = ts[name]
    X, y, idxs = make_lag_features(s, WINDOW)
    mask_train = idxs <= train.index[-1]
    mask_test = idxs > train.index[-1]
    X_train, y_train = X[mask_train], y[mask_train]
    X_test, y_test = X[mask_test], y[mask_test]

    # RandomForest
    rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    ypred_rf = rf.predict(X_test)
    rmse_rf = mean_squared_error(y_test, ypred_rf, squared=False)

    # GradientBoosting
    gb = GradientBoostingRegressor(n_estimators=200, random_state=42)
    gb.fit(X_train, y_train)
    ypred_gb = gb.predict(X_test)
    rmse_gb = mean_squared_error(y_test, ypred_gb, squared=False)

    results.append({'series': name, 'rf_rmse': float(rmse_rf), 'gb_rmse': float(rmse_gb)})
    print(f"{name}: RF RMSE={rmse_rf:.2f}, GB RMSE={rmse_gb:.2f}")

    # Retrain on full data for forecasting
    X_full, y_full, idx_full = make_lag_features(s, WINDOW)
    rf_full = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
    rf_full.fit(X_full, y_full)
    gb_full = GradientBoostingRegressor(n_estimators=300, random_state=42)
    gb_full.fit(X_full, y_full)
    joblib.dump(rf_full, os.path.join(MODEL_DIR, f"rf_{name}.joblib"))
    joblib.dump(gb_full, os.path.join(MODEL_DIR, f"gb_{name}.joblib"))

    # recursive forecasting
    future_idx = pd.date_range(start=ts.index[-1] + pd.Timedelta(days=1), periods=FORECAST_HORIZON, freq='D')
    last_window = s.values[-WINDOW:].astype(float).copy()
    rf_preds = []
    gb_preds = []
    curr_rf = last_window.copy()
    curr_gb = last_window.copy()
    for _ in range(FORECAST_HORIZON):
        next_date = future_idx[len(rf_preds)]
        cal = np.array([next_date.weekday(), next_date.month, 1 if next_date.weekday()>=5 else 0])
        xr = np.concatenate([curr_rf, cal]).reshape(1, -1)
        p_rf = rf_full.predict(xr)[0]
        p_rf = max(0, p_rf)
        rf_preds.append(p_rf)
        curr_rf = np.roll(curr_rf, -1)
        curr_rf[-1] = p_rf

        xg = np.concatenate([curr_gb, cal]).reshape(1, -1)
        p_gb = gb_full.predict(xg)[0]
        p_gb = max(0, p_gb)
        gb_preds.append(p_gb)
        curr_gb = np.roll(curr_gb, -1)
        curr_gb[-1] = p_gb

    rf_forecasts[name] = pd.Series(rf_preds, index=future_idx)
    gb_forecasts[name] = pd.Series(gb_preds, index=future_idx)

# Save forecasts
rf_df = pd.DataFrame(rf_forecasts)
gb_df = pd.DataFrame(gb_forecasts)
rf_df.to_csv("/mnt/data/rf_60day_forecast.csv")
gb_df.to_csv("/mnt/data/gb_60day_forecast.csv")

# Summary table
res_df = pd.DataFrame(results).set_index('series').sort_values('rf_rmse')
display_dataframe_to_user("ML test RMSE comparison (lower better)", res_df)
res_df.to_csv("/mnt/data/ml_test_rmse_summary.csv")

print("Saved forecasts to /mnt/data/rf_60day_forecast.csv and /mnt/data/gb_60day_forecast.csv")
print("Saved RMSE summary to /mnt/data/ml_test_rmse_summary.csv")

# Plot the last year of actuals + forecasts for TOTAL series as an example
series_plot = 'TOTAL'
plt.figure(figsize=(10,4))
plt.plot(ts[series_plot].last('400D').index, ts[series_plot].last('400D').values, label='Actual (last ~400d)')
plt.plot(rf_df.index, rf_df[series_plot].values, label='RF forecast')
plt.plot(gb_df.index, gb_df[series_plot].values, label='GB forecast')
plt.legend()
plt.title(f"Actual (last ~400 days) + 60-day forecasts for {series_plot}")
plt.xlabel("Date")
plt.ylabel("Daily counts")
plt.show()

# Display top rows of forecast files
display_dataframe_to_user("RF 60-day forecast (first 10 rows)", rf_df.head(10))
display_dataframe_to_user("GB 60-day forecast (first 10 rows)", gb_df.head(10))

print("[Download RF forecast CSV](/mnt/data/rf_60day_forecast.csv)")
print("[Download GB forecast CSV](/mnt/data/gb_60day_forecast.csv)")


TypeError: exists: path should be string, bytes, os.PathLike or integer, not DataFrame